In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    accuracy_score,
    auc,
    average_precision_score,
    classification_report,
    confusion_matrix,
    precision_recall_curve,
    roc_auc_score,
    roc_curve,
    f1_score,
    make_scorer
)
from sklearn.model_selection import (
    cross_val_predict,
    cross_val_score,
    train_test_split,
    StratifiedKFold,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, RobustScaler, StandardScaler
from sklearn import svm
import optuna
import joblib

project_root = "D:/Projects/Summer-2024-ECE-597-Group8-main"
data_path = "D:/Projects/Summer-2024-ECE-597-Group8-main/Summer-2024-ECE-597-Group8-main/data/processed/features_tfidf_labels.csv"
model_path_template = "D:/Projects/Summer-2024-ECE-597-Group8-main/Summer-2024-ECE-597-Group8-main/data/models/svm_model_trial_{}.pkl"
random_state = 42
test_size = 0.1

df = pd.read_csv(data_path)
df = df.drop(columns=["js_code"])

def log_transform(x):
    return np.log1p(x)

pipeline_log_transform_RobustScaler = Pipeline(
    steps=[
        ("log_transform", FunctionTransformer(log_transform)),
        ("RobustScaler", RobustScaler()),
    ]
)

features_preprocessor = ColumnTransformer(
    transformers=[
        (
            "log_transform_RobustScaler",
            pipeline_log_transform_RobustScaler,
            [
                "Word_Count",
                "Homoglyphs",
                "Total_Abnormal_Count",
                "html_tags",
            ],
        ),
    ],
    remainder="passthrough",
)

X = features_preprocessor.fit_transform(df.drop(columns=["Label"]))
y = df["Label"].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state
)

def objective(trial):
    svc_c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
    gamma = trial.suggest_float('gamma', 1e-10, 1e1, log=True) if kernel in ['poly', 'rbf', 'sigmoid'] else 'scale'
    class_weight = trial.suggest_categorical('class_weight', [None, 'balanced'])
    
    classifier = svm.SVC(C=svc_c, kernel=kernel, degree=degree, gamma=gamma, class_weight=class_weight)

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', classifier)
    ])

    f1_scorer = make_scorer(f1_score, pos_label=1)
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring=f1_scorer, n_jobs=-1)

    # Save the model after each trial
    model_path = model_path_template.format(trial.number)
    joblib.dump(pipeline, model_path)
    print(f"Model saved to {model_path}")

    return np.mean(scores)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40, n_jobs=6)

print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Train final model with best hyperparameters
best_params = study.best_params
best_svm = svm.SVC(C=best_params['svc_c'], kernel=best_params['kernel'], degree=best_params.get('degree', 3), gamma=best_params.get('gamma', 'scale'), class_weight=best_params['class_weight'])

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', best_svm)
])

pipeline.fit(X_train, y_train)

# Save the best model
final_model_path = model_path_template.format('best')
joblib.dump(pipeline, final_model_path)
print(f"Best model saved to {final_model_path}")

# Evaluate on the test set
y_pred = pipeline.predict(X_test)
report = classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1'])
print("Classification Report:\n")
print(report)


[I 2024-07-05 23:22:30,283] A new study created in memory with name: no-name-a04badbb-d6d4-4bbe-ac05-13bd7819a66f
[I 2024-07-05 23:24:07,983] Trial 1 finished with value: 0.6501213680000735 and parameters: {'svc_c': 5834987.2842162, 'kernel': 'sigmoid', 'gamma': 1.438641964596868e-09, 'class_weight': 'balanced'}. Best is trial 1 with value: 0.6501213680000735.


Model saved to D:/Projects/Summer-2024-ECE-597-Group8-main/Summer-2024-ECE-597-Group8-main/data/models/svm_model_trial_1.pkl


[I 2024-07-05 23:25:08,194] Trial 3 finished with value: 0.0 and parameters: {'svc_c': 0.0003070893585264844, 'kernel': 'sigmoid', 'gamma': 0.006097959483018337, 'class_weight': None}. Best is trial 1 with value: 0.6501213680000735.


Model saved to D:/Projects/Summer-2024-ECE-597-Group8-main/Summer-2024-ECE-597-Group8-main/data/models/svm_model_trial_3.pkl


[I 2024-07-05 23:25:16,278] Trial 4 finished with value: 0.8486180331130496 and parameters: {'svc_c': 734001.0289799175, 'kernel': 'rbf', 'gamma': 5.2313492629576304e-05, 'class_weight': None}. Best is trial 4 with value: 0.8486180331130496.


...

Model saved to D:/Projects/Summer-2024-ECE-597-Group8-main/Summer-2024-ECE-597-Group8-main/data/models/svm_model_trial_2.pkl


[I 2024-07-06 01:25:19,442] Trial 5 finished with value: 0.5903489618477477 and parameters: {'svc_c': 0.10518955212511016, 'kernel': 'poly', 'degree': 5, 'gamma': 1.0393456944375061, 'class_weight': None}. Best is trial 4 with value: 0.8486180331130496.


Model saved to D:/Projects/Summer-2024-ECE-597-Group8-main/Summer-2024-ECE-597-Group8-main/data/models/svm_model_trial_5.pkl


[I 2024-07-06 01:27:52,844] Trial 8 finished with value: 0.4004465208956778 and parameters: {'svc_c': 7614649.981426936, 'kernel': 'rbf', 'gamma': 0.16763769603155165, 'class_weight': None}. Best is trial 4 with value: 0.8486180331130496.


Model saved to D:/Projects/Summer-2024-ECE-597-Group8-main/Summer-2024-ECE-597-Group8-main/data/models/svm_model_trial_8.pkl
